Goal: Review the `~/utils/metakg/parser.py`, discover what keys are missing from bte.

In [8]:
# Import libraries
import yaml
import warnings
import pprint

from controller.smartapi import SmartAPI
from utils.metakg import parser


# Ignore all warnings
warnings.filterwarnings("ignore")


In [9]:
mkg_parser = parser.MetaKGParser()

In [3]:
# # bte_keys = ["query_operation", "response_mapping"] 
# operations_keys = [
#     "supportBatch",  # boolean
#     "useTemplating",  # boolean
#     "inputs",  # array with objects that have required "id" and "semantic" fields
#     "requestBody",  # object with required "body" field
#     "outputs",  # array with objects that have required "id" and "semantic" fields
#   a  "predicate",  # string
#     "response_mapping"  # object (references $defs/responseMapping)
# ]


# Non-TRAPI Parser

First draft

In [7]:
# Initialize sets for the keys
metakg_nontrapi_bte_query_keys = set()
metakg_nontrapi_bte_response_keys = set()

for smartapi in SmartAPI.get_all(10):
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Capture raw data for the 'bte' part in the raw SmartAPI doc
    raw_bte_data = None


    # Get non-TRAPI and TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)
    metakg2 = mkg_parser.get_TRAPI_metadatas(data=source_data)

    # Process non-TRAPI metadata
    if metakg:
        print(source_data['components'].keys())
        raw_bte_data_op = source_data['components']['x-bte-kgs-operations']
        raw_bte_data_map=  source_data['components']['x-bte-response-mapping']
        print(f"SmartAPI (non-TRAPI): {smartapi._id}")
        # print(len(metakg))
        for edge in metakg:
            if 'bte' in edge['api']:
                # Separate query_operation and response_mapping keys into different sets
                metakg_nontrapi_bte_query_keys.update(edge['api']['bte']['query_operation'].keys())
                metakg_nontrapi_bte_response_keys.update(edge['api']['bte']['response_mapping'].keys())
                # print(edge['api']['bte']['query_operation'])
                # print(edge['api']['bte']['response_mapping'])

    # # Process TRAPI metadata
    if metakg2:
        print(f"SmartAPI (TRAPI): {smartapi._id}")
        print(len(metakg2))
        ct=0
        for edge in metakg2:
            while ct<2:
                ct+=1
                if 'bte' in edge['api']:
                    # Separate query_operation and response_mapping keys into different sets
                    metakg_trapi_bte_query_keys.update(edge['api']['bte']['query_operation'].keys())
                    print(edge['api']['bte']['query_operation'].keys())
                    # metakg_trapi_bte_response_keys.update(edge['api']['bte']['response_mapping'].keys())

# Output the collected keys
print("Non-TRAPI BTE Query keys:", metakg_nontrapi_bte_query_keys)
print("Non-TRAPI BTE Response keys:", metakg_nontrapi_bte_response_keys)
# print("TRAPI BTE Query keys:", metakg_trapi_bte_query_keys)
# print("TRAPI BTE Response keys:", metakg_trapi_bte_response_keys)


dict_keys(['schemas'])
dict_keys(['schemas'])
dict_keys(['schemas'])
dict_keys(['schemas'])
SmartAPI (TRAPI): 0991bc29489a8d918ff6035fd38b0ccc
19098


NameError: name 'metakg_trapi_bte_query_keys' is not defined

Methods

In [15]:

mkg_non_trapi_dict = {}
for smartapi in SmartAPI.get_all(20):

    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        print()
        print(f"--------------------- HAS METAKG: {smartapi._id} ------------------------------------\n")
        mkg_non_trapi_dict[smartapi._id] = {}


--------------------- HAS METAKG: edeb26858bd27d0322af93e7a9e08761 ------------------------------------


--------------------- HAS METAKG: 1138c3297e8e403b6ac10cff5609b319 ------------------------------------



In [79]:
len(mkg_non_trapi_dict)

47

In [150]:
import re
import yaml

# Helper function to normalize key names (convert camelCase to snake_case and lowercase everything)
def normalize_key(key):
    key = re.sub(r'(?<!^)(?=[A-Z])', '_', key).lower()
    return key

# Define the method to process a SmartAPI and update mkg_non_trapi_dict
def process_smartapi(smartapi_id, mkg_non_trapi_dict):
    # Fetch the SmartAPI record by its ID
    smartapi = SmartAPI.get(smartapi_id)
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        # Initialize the comparison structure in mkg_non_trapi_dict for this SmartAPI
        mkg_non_trapi_dict[smartapi._id] = {
            'query': {
                'raw_bte_query_keys': {},
                'parsed_bte_query_keys': set(),
                'missing_bte_query_keys': {},
                'similar_bte_query_keys': {}
            },
            'response': {
                'raw_bte_response_keys': {},
                'parsed_bte_response_keys': set(),
                'missing_bte_response_keys': {},
                'similar_bte_response_keys': {}
            }
        }

        # Capture raw data for query_operation and response_mapping from the raw SmartAPI doc
        if 'components' in source_data:
            raw_bte_data_op = source_data['components'].get('x-bte-kgs-operations', {})
            raw_bte_data_map = source_data['components'].get('x-bte-response-mapping', {})

            # Collect raw query keys and normalize them
            for key, value in raw_bte_data_op.items():
                if isinstance(value, list) and value:  # Ensure it's a list with content
                    mkg_non_trapi_dict[smartapi._id]['query']['raw_bte_query_keys'][key] = {
                        normalize_key(k) for k in value[0].keys()
                    }

            # Collect raw response keys without normalization
            for key, value in raw_bte_data_map.items():
                if isinstance(value, dict):  # Ensure it's a dict with content
                    mkg_non_trapi_dict[smartapi._id]['response']['raw_bte_response_keys'][key] = {
                        k for k in value.keys()  # No normalization here
                    }

        # Process the parsed metadata (non-TRAPI)
        for edge in metakg:
            if 'bte' in edge['api']:
                # Collect parsed query keys and normalize them
                mkg_non_trapi_dict[smartapi._id]['query']['parsed_bte_query_keys'].update({
                    normalize_key(k) for k in edge['api']['bte']['query_operation'].keys()
                })
                # Collect parsed response keys without normalization
                if 'response_mapping' in edge['api']['bte']:
                    for key, value in edge['api']['bte']['response_mapping'].items():
                        if isinstance(value, dict):
                            mkg_non_trapi_dict[smartapi._id]['response']['parsed_bte_response_keys'].update({
                                k for k in value.keys()
                            })
                        else:
                            mkg_non_trapi_dict[smartapi._id]['response']['parsed_bte_response_keys'].add(key)

        # Compare and find missing and similar keys for query keys
        for raw_key, raw_value_keys in mkg_non_trapi_dict[smartapi._id]['query']['raw_bte_query_keys'].items():
            parsed_keys = mkg_non_trapi_dict[smartapi._id]['query']['parsed_bte_query_keys']
            missing_keys = raw_value_keys - parsed_keys
            similar_keys = raw_value_keys & parsed_keys
            mkg_non_trapi_dict[smartapi._id]['query']['missing_bte_query_keys'][raw_key] = missing_keys
            mkg_non_trapi_dict[smartapi._id]['query']['similar_bte_query_keys'][raw_key] = similar_keys

        # Compare and find missing and similar keys for response keys
        for raw_key, raw_value_keys in mkg_non_trapi_dict[smartapi._id]['response']['raw_bte_response_keys'].items():
            parsed_keys = mkg_non_trapi_dict[smartapi._id]['response']['parsed_bte_response_keys']
            missing_keys = raw_value_keys - parsed_keys
            similar_keys = raw_value_keys & parsed_keys
            mkg_non_trapi_dict[smartapi._id]['response']['missing_bte_response_keys'][raw_key] = missing_keys
            mkg_non_trapi_dict[smartapi._id]['response']['similar_bte_response_keys'][raw_key] = similar_keys


# Function to generate the comparison report
def generate_comparison_report(smartapi_id, smartapi_comparison):
    """
    Generates and prints a report for a specific SmartAPI comparison data.
    
    :param smartapi_id: The ID of the SmartAPI document
    :param smartapi_comparison: Dictionary containing comparison data for SmartAPIs
    """
    # Fetch the comparison data for the given SmartAPI ID
    data = smartapi_comparison[smartapi_id]
    
    # Clean, organized report output
    print(f"SmartAPI ID: {smartapi_id}")
    print("=" * 40)
    
    # Query Operation Comparison (print one example)
    print("Query Operation Comparison:")
    if data['query']['missing_bte_query_keys']:  # Ensure there's at least one item
        raw_key, missing_keys = next(iter(data['query']['missing_bte_query_keys'].items()))
        raw_keys = ", ".join(data['query']['raw_bte_query_keys'][raw_key])
        parsed_keys = ", ".join(data['query']['parsed_bte_query_keys'])
        missing_keys_str = ", ".join(missing_keys) if missing_keys else "NONE MISSING"
        similar_keys = ", ".join(data['query']['similar_bte_query_keys'][raw_key]) if data['query']['similar_bte_query_keys'][raw_key] else "NO SIMILAR KEYS"
        
        print(f"  Identifier: {raw_key}")
        print(f"    Raw BTE Query Keys: [{raw_keys}]")
        print(f"    Parsed BTE Query Keys: [{parsed_keys}]")
        print(f"    Missing BTE Query Keys: [{missing_keys_str}]")
        print(f"    Similar BTE Query Keys: [{similar_keys}]")
    
    print("-" * 40)
    
    # Response Mapping Comparison (print one example)
    print("\nResponse Mapping Comparison:")
    if data['response']['missing_bte_response_keys']:  # Ensure there's at least one item
        raw_key, missing_keys = next(iter(data['response']['missing_bte_response_keys'].items()))
        raw_keys = ", ".join(data['response']['raw_bte_response_keys'][raw_key])
        parsed_keys = ", ".join(data['response']['parsed_bte_response_keys'])
        missing_keys_str = ", ".join(missing_keys) if missing_keys else "NONE MISSING"
        similar_keys = ", ".join(data['response']['similar_bte_response_keys'][raw_key]) if data['response']['similar_bte_response_keys'][raw_key] else "NO SIMILAR KEYS"
        
        print(f"  Identifier: {raw_key}")
        print(f"    Raw BTE Response Keys: [{raw_keys}]")
        print(f"    Parsed BTE Response Keys: [{parsed_keys}]")
        print(f"    Missing BTE Response Keys: [{missing_keys_str}]")
        print(f"    Similar BTE Response Keys: [{similar_keys}]")
    
    print("=" * 40)

Get the initial values -

In [127]:
sample_id = next(iter(mkg_non_trapi_dict.keys()))
smartapi = SmartAPI.get(sample_id)
raw_data = smartapi.';';
raw.decode('utf-8')
source_data = yaml.safe_load(raw_data)

In [130]:
# Process SmartAPI comparison and store results in the dictionary
process_smartapi(sample_id, mkg_non_trapi_dict)

# Call the comparison report function for the given sample_id
generate_comparison_report(sample_id, mkg_non_trapi_dict)


SmartAPI ID: edeb26858bd27d0322af93e7a9e08761
Query Operation Comparison:
  Identifier: gene-chem
    Raw BTE Query Keys: [predicate, knowledge_level, agent_type, use_templating, request_body, support_batch, test_examples, outputs, parameters, response_mapping, inputs, source]
    Parsed BTE Query Keys: [path, request_body, support_batch, method, params, server]
    Missing BTE Query Keys: [predicate, knowledge_level, agent_type, use_templating, test_examples, outputs, inputs, response_mapping, parameters, source]
    Similar BTE Query Keys: [request_body, support_batch]
----------------------------------------

Response Mapping Comparison:
  Identifier: chem
    Raw BTE Response Keys: [ref_url, MESH, ref_pmcid]
    Parsed BTE Response Keys: [ref_url, MESH, ref_pmcid, NCBIGene]
    Missing BTE Response Keys: [NONE MISSING]
    Similar BTE Response Keys: [MESH, ref_url, ref_pmcid]


In [100]:
# Method to inspect raw document and report missing keys
def inspect_raw_document(smartapi_id, smartapi_comparison, source_data):
    """
    Inspects the raw SmartAPI document and compares the keys to identify any missing keys
    that are not captured by the parser.

    :param smartapi_id: The ID of the SmartAPI document
    :param smartapi_comparison: Dictionary containing comparison data for SmartAPIs
    :param source_data: The raw source data (JSON or YAML) of the SmartAPI document
    :return: A report of missing keys in the raw document not parsed by the parser
    """
    # Fetch the comparison data for the given SmartAPI ID
    data = smartapi_comparison.get(smartapi_id, {})
    
    missing_keys_report = {
        'query': {},
        'response': {}
    }

    # Check in the raw document for query operation missing keys
    raw_bte_data_op = source_data.get('components', {}).get('x-bte-kgs-operations', {})
    for raw_key, raw_value in raw_bte_data_op.items():
        if isinstance(raw_value, list) and raw_value:  # Ensure it's a list with content
            raw_keys_set = {normalize_key(k) for k in raw_value[0].keys()}  # Normalized keys from raw doc
            parsed_keys_set = data['query']['parsed_bte_query_keys'] if 'query' in data else set()

            # Identify missing keys that are in raw document but not in parsed output
            missing_in_parsed = raw_keys_set - parsed_keys_set
            if missing_in_parsed:
                missing_keys_report['query'][raw_key] = list(missing_in_parsed)
    
    # Check in the raw document for response mapping missing keys
    raw_bte_data_map = source_data.get('components', {}).get('x-bte-response-mapping', {})
    for raw_key, raw_value in raw_bte_data_map.items():
        if isinstance(raw_value, dict):  # Ensure it's a dict with content
            raw_keys_set = set(raw_value.keys())  # Do not normalize for response keys
            parsed_keys_set = data['response']['parsed_bte_response_keys'] if 'response' in data else set()

            # Identify missing keys that are in raw document but not in parsed output
            missing_in_parsed = raw_keys_set - parsed_keys_set
            if missing_in_parsed:
                missing_keys_report['response'][raw_key] = list(missing_in_parsed)

    return missing_keys_report


In [102]:
# Call the method to inspect the raw document and check for missing keys
missing_keys_report = inspect_raw_document(sample_id, mkg_non_trapi_dict, source_data)

# Output the missing keys report
print(f"Missing Keys Report for SmartAPI ID: {sample_id}")
pprint.pprint(missing_keys_report)

Missing Keys Report for SmartAPI ID: edeb26858bd27d0322af93e7a9e08761
{'query': {'chem-disease': ['predicate',
                            'knowledge_level',
                            'agent_type',
                            'use_templating',
                            'test_examples',
                            'outputs',
                            'inputs',
                            'response_mapping',
                            'parameters',
                            'source'],
           'chem-gene': ['predicate',
                         'knowledge_level',
                         'agent_type',
                         'use_templating',
                         'test_examples',
                         'outputs',
                         'inputs',
                         'response_mapping',
                         'parameters',
                         'source'],
           'disease-chem': ['predicate',
                            'knowledge_level',
                  

Reporting

In [4]:
import re

# Helper function to normalize key names (convert camelCase to snake_case and lowercase everything)
def normalize_key(key):
    # Convert from camelCase to snake_case
    key = re.sub(r'(?<!^)(?=[A-Z])', '_', key).lower()
    return key

# Initialize a dictionary to store the results for each SmartAPI
smartapi_comparison = {}

for smartapi in SmartAPI.get_all(10):
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        # Initialize dictionary for this SmartAPI entry
        smartapi_comparison[smartapi._id] = {
            'query': {
                'raw_bte_query_keys': {},
                'parsed_bte_query_keys': set(),
                'missing_bte_query_keys': {},
                'similar_bte_query_keys': {}
            },
            'response': {
                'raw_bte_response_keys': {},
                'parsed_bte_response_keys': set(),
                'missing_bte_response_keys': {},
                'similar_bte_response_keys': {}
            }
        }

        # Capture raw data for query_operation and response_mapping from the raw SmartAPI doc
        if 'components' in source_data:
            raw_bte_data_op = source_data['components'].get('x-bte-kgs-operations', {})
            raw_bte_data_map = source_data['components'].get('x-bte-response-mapping', {})

            # Collect raw query keys and normalize them (e.g., inside 'disease-chem')
            for key, value in raw_bte_data_op.items():
                if isinstance(value, list) and value:  # Ensure it's a list with content
                    smartapi_comparison[smartapi._id]['query']['raw_bte_query_keys'][key] = {
                        normalize_key(k) for k in value[0].keys()
                    }

            # Collect raw response keys without normalization, with nested key checks
            for key, value in raw_bte_data_map.items():
                if isinstance(value, dict):  # Ensure it's a dict with content
                    smartapi_comparison[smartapi._id]['response']['raw_bte_response_keys'][key] = {
                        k for k in value.keys()  # No normalization here
                    }

        # Process the parsed metadata (non-TRAPI)
        for edge in metakg:
            if 'bte' in edge['api']:
                # Collect parsed query keys and normalize them
                smartapi_comparison[smartapi._id]['query']['parsed_bte_query_keys'].update({
                    normalize_key(k) for k in edge['api']['bte']['query_operation'].keys()
                })
                # Collect parsed response keys without normalization, with nested key checks
                if 'response_mapping' in edge['api']['bte']:
                    for key, value in edge['api']['bte']['response_mapping'].items():
                        if isinstance(value, dict):  # Ensure it's a dictionary and go a level down
                            smartapi_comparison[smartapi._id]['response']['parsed_bte_response_keys'].update({
                                k for k in value.keys()  # No normalization here
                            })
                        else:
                            # If value isn't a dictionary, just collect the top-level keys without normalization
                            smartapi_comparison[smartapi._id]['response']['parsed_bte_response_keys'].add(key)

        # Compare and find missing and similar keys for each identifier (e.g., disease-chem) in query keys
        for raw_key, raw_value_keys in smartapi_comparison[smartapi._id]['query']['raw_bte_query_keys'].items():
            parsed_keys = smartapi_comparison[smartapi._id]['query']['parsed_bte_query_keys']
            missing_keys = raw_value_keys - parsed_keys
            similar_keys = raw_value_keys & parsed_keys
            smartapi_comparison[smartapi._id]['query']['missing_bte_query_keys'][raw_key] = missing_keys
            smartapi_comparison[smartapi._id]['query']['similar_bte_query_keys'][raw_key] = similar_keys

        # Compare and find missing and similar keys for each identifier in response keys
        for raw_key, raw_value_keys in smartapi_comparison[smartapi._id]['response']['raw_bte_response_keys'].items():
            parsed_keys = smartapi_comparison[smartapi._id]['response']['parsed_bte_response_keys']
            missing_keys = raw_value_keys - parsed_keys
            similar_keys = raw_value_keys & parsed_keys
            smartapi_comparison[smartapi._id]['response']['missing_bte_response_keys'][raw_key] = missing_keys
            smartapi_comparison[smartapi._id]['response']['similar_bte_response_keys'][raw_key] = similar_keys

# Output the comparison results
for smartapi_id, data in smartapi_comparison.items():
    print(f"SmartAPI ID: {smartapi_id}")

    print("-" * 40)

    print("Query Operation Comparison:")
    for raw_key, missing_keys in data['query']['missing_bte_query_keys'].items():
        print(f"  Identifier: {raw_key}")
        print(f"    Raw BTE Query Keys: {data['query']['raw_bte_query_keys'][raw_key]}")
        print(f"    Parsed BTE Query Keys: {data['query']['parsed_bte_query_keys']}")
        print(f"    Missing BTE Query Keys: {missing_keys}")
        print(f"    Similar BTE Query Keys: {data['query']['similar_bte_query_keys'][raw_key]}")

    print("-" * 40)
    print("\nResponse Mapping Comparison:")
    for raw_key, missing_keys in data['response']['missing_bte_response_keys'].items():
        print(f"  Identifier: {raw_key}")
        print(f"    Raw BTE Response Keys: {data['response']['raw_bte_response_keys'][raw_key]}")
        print(f"    Parsed BTE Response Keys: {data['response']['parsed_bte_response_keys']}")
        print(f"    Missing BTE Response Keys: {missing_keys}")
        print(f"    Similar BTE Response Keys: {data['response']['similar_bte_response_keys'][raw_key]}")
    
    print("\n" + "-" * 40 + "\n")  # Adds a space between each report



SmartAPI ID: edeb26858bd27d0322af93e7a9e08761
----------------------------------------
Query Operation Comparison:
  Identifier: gene-chem
    Raw BTE Query Keys: {'test_examples', 'source', 'predicate', 'agent_type', 'request_body', 'support_batch', 'inputs', 'knowledge_level', 'use_templating', 'parameters', 'response_mapping', 'outputs'}
    Parsed BTE Query Keys: {'server', 'request_body', 'support_batch', 'params', 'method', 'path'}
    Missing BTE Query Keys: {'test_examples', 'source', 'predicate', 'agent_type', 'inputs', 'knowledge_level', 'use_templating', 'parameters', 'response_mapping', 'outputs'}
    Similar BTE Query Keys: {'request_body', 'support_batch'}
  Identifier: chem-gene
    Raw BTE Query Keys: {'test_examples', 'source', 'predicate', 'agent_type', 'request_body', 'support_batch', 'inputs', 'knowledge_level', 'use_templating', 'parameters', 'response_mapping', 'outputs'}
    Parsed BTE Query Keys: {'server', 'request_body', 'support_batch', 'params', 'method', 'p

In [ ]:
import re
from collections import Counter

# Helper function to normalize key names (convert camelCase to snake_case and lowercase everything)
def normalize_key(key):
    # Convert from camelCase to snake_case
    key = re.sub(r'(?<!^)(?=[A-Z])', '_', key).lower()
    return key

# Initialize overall stats
overall_stats = {
    'query': {
        'raw_bte_query_keys': Counter(),
        'parsed_bte_query_keys': Counter(),
        'missing_bte_query_keys': Counter(),
        'similar_bte_query_keys': Counter()
    },
    'response': {
        'raw_bte_response_keys': Counter(),
        'parsed_bte_response_keys': Counter(),
        'missing_bte_response_keys': Counter(),
        'similar_bte_response_keys': Counter()
    }
}

# Track the number of documents processed
total_docs_processed = 0

for smartapi in SmartAPI.get_all(1000):  # Modify to get more docs as needed
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        total_docs_processed += 1

        # Capture raw data for query_operation and response_mapping from the raw SmartAPI doc
        if 'components' in source_data:
            raw_bte_data_op = source_data['components'].get('x-bte-kgs-operations', {})
            raw_bte_data_map = source_data['components'].get('x-bte-response-mapping', {})

            # Collect raw query keys and normalize them (e.g., inside 'disease-chem')
            for key, value in raw_bte_data_op.items():
                if isinstance(value, list) and value:  # Ensure it's a list with content
                    raw_keys = {normalize_key(k) for k in value[0].keys()}
                    overall_stats['query']['raw_bte_query_keys'].update(raw_keys)

            # Collect raw response keys without normalization
            for key, value in raw_bte_data_map.items():
                if isinstance(value, dict):  # Ensure it's a dict with content
                    raw_response_keys = {k for k in value.keys()}  # No normalization
                    overall_stats['response']['raw_bte_response_keys'].update(raw_response_keys)

        # Process the parsed metadata (non-TRAPI)
        for edge in metakg:
            if 'bte' in edge['api']:
                # Collect parsed query keys and normalize them
                parsed_query_keys = {normalize_key(k) for k in edge['api']['bte']['query_operation'].keys()}
                overall_stats['query']['parsed_bte_query_keys'].update(parsed_query_keys)

                # Collect parsed response keys without normalization
                if 'response_mapping' in edge['api']['bte']:
                    for key, value in edge['api']['bte']['response_mapping'].items():
                        if isinstance(value, dict):
                            parsed_response_keys = {k for k in value.keys()}  # No normalization
                            overall_stats['response']['parsed_bte_response_keys'].update(parsed_response_keys)

        # Compare and accumulate missing and similar keys for query
        for raw_key in overall_stats['query']['raw_bte_query_keys'].keys():
            missing_keys = overall_stats['query']['raw_bte_query_keys'] - overall_stats['query']['parsed_bte_query_keys']
            similar_keys = overall_stats['query']['raw_bte_query_keys'] & overall_stats['query']['parsed_bte_query_keys']
            overall_stats['query']['missing_bte_query_keys'].update(missing_keys)
            overall_stats['query']['similar_bte_query_keys'].update(similar_keys)

        # Compare and accumulate missing and similar keys for response
        for raw_key in overall_stats['response']['raw_bte_response_keys'].keys():
            missing_keys = overall_stats['response']['raw_bte_response_keys'] - overall_stats['response']['parsed_bte_response_keys']
            similar_keys = overall_stats['response']['raw_bte_response_keys'] & overall_stats['response']['parsed_bte_response_keys']
            overall_stats['response']['missing_bte_response_keys'].update(missing_keys)
            overall_stats['response']['similar_bte_response_keys'].update(similar_keys)

# Output overall statistics
print(f"Total SmartAPI documents processed: {total_docs_processed}")

print("\nOverall Query Operation Statistics:")

The TRAPI parser:

In [67]:

# Helper function to normalize key names (convert camelCase to snake_case and lowercase everything)
def normalize_key(key):
    # Convert from camelCase to snake_case
    key = re.sub(r'(?<!^)(?=[A-Z])', '_', key).lower()
    return key

for smartapi in SmartAPI.get_all(10):
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    
    # Get TRAPI metakg data
    metakg2 = mkg_parser.get_TRAPI_metadatas(data=source_data)

    if metakg2:
        # Capture raw data for query_operation and response_mapping from the raw SmartAPI doc
        if 'components' in source_data:
            raw_bte_data_op = source_data['components'].get('x-bte-kgs-operations', {})

            # # Collect raw query keys and normalize them (e.g., inside 'disease-chem')
            # for key, value in raw_bte_data_op.items():
            #     if isinstance(value, list) and value:  # Ensure it's a list with content
            #         raw_keys = {normalize_key(k) for k in value[0].keys()}
            #         overall_stats['query']['raw_bte_query_keys'].update(raw_keys)
        ct = 0
        for edge in metakg2:
            while ct < 2:
                ct += 1                
                parsed_edge_keys = edge['api']['bte']['query_operation'].keys()

            print("Original doc keys: ", )

dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query
dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query
dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query
dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query
dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query
dict_keys(['path', 'method', 'server', 'path_params', 'params', 'request_body', 'support_batch', 'input_separator'])
/query


---

In [1]:
# analysis methods 
import pprint

def load_and_parse_smartapi_data(sample_id):
    smartapi = SmartAPI.get(sample_id)
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)
    return source_data

def analyze_smartapi_query_raw(sample_id):
    source_data = load_and_parse_smartapi_data(sample_id)

    print("Query Operation Example:")
    kg_ops_data = source_data['components']['x-bte-kgs-operations']
    pprint.pprint(kg_ops_data)
    # kg_ops_example = next(iter(kg_ops_data.values()))
    # pprint.pprint(kg_ops_example[0])

def analyze_smartapi_response_raw(sample_id):
    source_data = load_and_parse_smartapi_data(sample_id)

    res_map_data = source_data['components']['x-bte-response-mapping']
    print("Response Mapping Example:")
    pprint.pprint(res_map_data)

def view_metakg_from_parser(smartapi_id):
    source_data = load_and_parse_smartapi_data(smartapi_id)

    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)
    
    pprint.pprint(metakg)
    return metakg


In [2]:
# List of required properties
required_properties = ["supportBatch", "useTemplating", "inputs", "outputs", "predicate", "response_mapping"]

# Function to check required properties
def check_required_properties(api_output, required_properties):
    found_properties = {
        "supportBatch": api_output['api']['bte']['query_operation'].get('support_batch', None),
        "useTemplating": api_output['api'].get('useTemplating', None),
        "predicate": api_output.get('predicate', None),
        "response_mapping": api_output['api']['bte'].get('response_mapping', None)
    }

    # Inputs and Outputs might need custom parsing, here's a basic check
    # Assuming 'inputs' and 'outputs' refer to some input/output concepts
    # They are not clearly defined in the provided structure, so we leave them as missing for now.
    found_properties["inputs"] = None  # Placeholder
    found_properties["outputs"] = None  # Placeholder

    # Check for extra properties
    extra_properties = set(api_output['api'].keys()).difference(required_properties)

    return found_properties, list(extra_properties)



In [5]:
analyze_smartapi_query_raw('38e9e5169a72aee3659c9ddba956790d')

Query Operation Example:
{'ligand-protein': [{'agent_type': 'manual_agent',
                     'inputs': [{'id': 'INCHIKEY',
                                 'semantic': 'SmallMolecule'}],
                     'knowledge_level': 'knowledge_assertion',
                     'outputs': [{'id': 'UniProtKB', 'semantic': 'Gene'}],
                     'parameters': {'fields': 'subject.uniprot.accession, '
                                              'subject.name,object.name,relation.pmid,relation.bindingdb_link,relation.article_doi',
                                    'size': 1000},
                     'predicate': 'physically_interacts_with',
                     'requestBody': {'body': {'q': '{{ queryInputs }}',
                                              'scopes': 'object.inchikey'}},
                     'response_mapping': {'$ref': '#/components/x-bte-response-mapping/uniprot-subject'},
                     'source': 'infores:bindingdb',
                     'supportBatch': True

In [6]:
analyze_smartapi_response_raw('38e9e5169a72aee3659c9ddba956790d')

Response Mapping Example:
{'pubchem-object': {'INCHIKEY': 'object.inchikey',
                    'input_name': 'subject.name',
                    'output_name': 'object.name',
                    'ref_doi': 'relation.article_doi',
                    'ref_pmid': 'relation.pmid',
                    'source_url': 'relation.bindingdb_link'},
 'uniprot-subject': {'UniProtKB': 'subject.uniprot.accession',
                     'input_name': 'object.name',
                     'output_name': 'subject.name',
                     'ref_doi': 'relation.article_doi',
                     'ref_pmid': 'relation.pmid',
                     'source_url': 'relation.bindingdb_link'}}


#### Inspection by document

In [8]:
smartapi_id

'edeb26858bd27d0322af93e7a9e08761'

https://raw.githubusercontent.com/NCATS-Tangerine/translator-api-registry/master/pfocr/smartapi.yaml

In [9]:
analyze_smartapi_query_raw(smartapi_id)

Query Operation Example:
{'chem-disease': [{'agent_type': 'image_processing_agent',
                   'inputs': [{'id': 'MESH', 'semantic': 'SmallMolecule'}],
                   'knowledge_level': 'not_provided',
                   'outputs': [{'id': 'MESH', 'semantic': 'Disease'}],
                   'parameters': {'fields': 'associatedWith.mentions.diseases.mesh, '
                                            'associatedWith.figureUrl,associatedWith.pmc',
                                  'size': 1000},
                   'predicate': 'occurs_together_in_literature_with',
                   'requestBody': {'body': {'q': '{{ queryInputs }}',
                                            'scopes': 'associatedWith.mentions.chemicals.mesh'}},
                   'response_mapping': {'$ref': '#/components/x-bte-response-mapping/disease'},
                   'source': 'infores:pfocr',
                   'supportBatch': True,
                   'testExamples': [{'oneOutput': 'MESH:D009369',
  

In [10]:
analyze_smartapi_response_raw(smartapi_id)

Response Mapping Example:
{'chem': {'MESH': 'associatedWith.mentions.chemicals.mesh',
          'ref_pmcid': 'associatedWith.pmc',
          'ref_url': 'associatedWith.figureUrl'},
 'disease': {'MESH': 'associatedWith.mentions.diseases.mesh',
             'ref_pmcid': 'associatedWith.pmc',
             'ref_url': 'associatedWith.figureUrl'},
 'gene': {'NCBIGene': 'associatedWith.mentions.genes.ncbigene',
          'ref_pmcid': 'associatedWith.pmc',
          'ref_url': 'associatedWith.figureUrl'}}


In [11]:
view_metakg_from_parser(smartapi_id)

[{'api': {'agent_type': 'image_processing_agent',
          'bte': {'query_operation': {'method': 'post',
                                      'params': {'fields': 'associatedWith.mentions.chemicals.mesh, '
                                                           'associatedWith.figureUrl,associatedWith.pmc',
                                                 'size': 1000},
                                      'path': '/query',
                                      'request_body': {'body': {'q': '{{ '
                                                                     'queryInputs '
                                                                     '}}',
                                                                'scopes': 'associatedWith.mentions.genes.ncbigene'}},
                                      'server': 'https://biothings.ncats.io/pfocr',
                                      'support_batch': True},
                  'response_mapping': {'occurs_together_in_literatur

---

Second Example

In [20]:
sample_id_2 = list(mkg_non_trapi_dict.keys())[-1]


In [22]:
print(sample_id_2)


1138c3297e8e403b6ac10cff5609b319


In [23]:
analyze_smartapi_query_raw(sample_id_2)

Query Operation Example:
{'disease-drug-approved': [{'agent_type': 'automated_agent',
                            'inputs': [{'id': 'UMLS', 'semantic': 'Disease'}],
                            'knowledge_level': 'knowledge_assertion',
                            'outputs': [{'id': 'DRUGBANK',
                                         'semantic': 'SmallMolecule'}],
                            'parameters': {'fields': 'drug.drugbank, '
                                                     'drug.name, '
                                                     'indication.name, '
                                                     'trials.status',
                                           'filter': 'trials.status:approved',
                                           'size': 1000},
                            'predicate': 'treated_by',
                            'requestBody': {'body': {'q': '{{ queryInputs | '
                                                          'rmPrefix() }}',
        

In [27]:
mkg_edges=view_metakg_from_parser(sample_id_2)

[{'api': {'agent_type': 'automated_agent',
          'bte': {'query_operation': {'method': 'post',
                                      'params': {'fields': 'indication.umls, '
                                                           'drug.name, '
                                                           'indication.name, '
                                                           'trials.status',
                                                 'filter': 'trials.status:approved',
                                                 'size': 1000},
                                      'path': '/query',
                                      'request_body': {'body': {'q': '{{ '
                                                                     'queryInputs '
                                                                     '| '
                                                                     'rmPrefix() '
                                                                     '}}'

In [31]:
# mkg_edges

In [54]:
# Check for required properties and extra ones
for edge in mkg_edges:
    keys_found_no_map=set()
    print("-"*40)
    keys_found_no_map.update(list(edge['api'].keys()))
    keys_found_no_map.update(list(edge['api']['bte']['query_operation'].keys()))

    # print(edge['api']['bte']['query_operation'].keys())
    # print("----")
    # print(edge['api']['bte']['response_mapping'])
    # print()
    print(keys_found_no_map)


----------------------------------------
{'bte', 'smartapi', 'server', 'tags', 'agent_type', 'request_body', 'support_batch', 'knowledge_level', 'useTemplating', 'testExamples', 'x-translator', 'provided_by', 'params', 'method', 'path', 'name'}
----------------------------------------
{'bte', 'smartapi', 'server', 'tags', 'agent_type', 'request_body', 'support_batch', 'knowledge_level', 'useTemplating', 'testExamples', 'x-translator', 'provided_by', 'params', 'method', 'path', 'name'}
----------------------------------------
{'bte', 'smartapi', 'server', 'tags', 'agent_type', 'request_body', 'support_batch', 'knowledge_level', 'useTemplating', 'testExamples', 'x-translator', 'provided_by', 'params', 'method', 'path', 'name'}
----------------------------------------
{'bte', 'smartapi', 'server', 'tags', 'agent_type', 'request_body', 'support_batch', 'knowledge_level', 'useTemplating', 'testExamples', 'x-translator', 'provided_by', 'params', 'method', 'path', 'name'}


Get the `query_operations` data

In [21]:
import yaml
from collections import defaultdict

mkg_non_trapi_dict = {}
keys_found_no_map_full_set = set()
key_frequency_dict = defaultdict(int)  # New dictionary to track frequency of keys

for smartapi in SmartAPI.get_all(1000):
    
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        for edge in metakg:
            edge_list = list(edge['api'].keys()) + list(edge['api']['bte']['query_operation'].keys())
            
            keys_found_no_map_full_set.update(edge_list)
            
            # Update the frequency of each key found in this document
            for key in edge_list:
                key_frequency_dict[key] += 1

            # Store metadata for each SmartAPI document
            mkg_non_trapi_dict[smartapi._id] = {
                'query_operation': {
                    'keys_found': edge_list, 
                    'key_ct': len(edge_list), 
                    'data': edge['api']['bte']['query_operation']
                }
            }





In [24]:
# Print or store the response mapping key frequencies
for key, count in sorted(key_frequency_dict.items()):
    print(f"{key}: {count}")

# Print the total number of documents processed
# print(f"Total number of documents processed: {doc_count}")

bte: 5754
input_separator: 18
method: 5754
name: 5754
params: 5752
path: 5754
path_params: 14
provided_by: 5754
request_body: 5665
server: 5754
smartapi: 5754
support_batch: 5670
tags: 5754
x-translator: 5754


In [4]:
keys_found_no_map_full_set

{'bte',
 'input_separator',
 'method',
 'name',
 'params',
 'path',
 'path_params',
 'provided_by',
 'request_body',
 'server',
 'smartapi',
 'support_batch',
 'tags',
 'x-translator'}

In [5]:
import itertools


In [73]:
# Print the first 2 items
n_items = dict(itertools.islice(mkg_non_trapi_dict.items(), 5))
pprint.pprint(n_items)

{'09c8782d9f4027712e65b95424adba79': {'keys_found': ['name',
                                                     'smartapi',
                                                     'tags',
                                                     'x-translator',
                                                     'provided_by',
                                                     'agent_type',
                                                     'knowledge_level',
                                                     'testExamples',
                                                     'useTemplating',
                                                     'bte',
                                                     'params',
                                                     'request_body',
                                                     'path',
                                                     'method',
                                                     'server',
                   

Get the `response-mapping` data

In [29]:
import re

In [19]:
import yaml
from collections import defaultdict
import re

mkg_non_trapi_dict = {}
keys_found_no_map_full_set = set()
response_mapping_frequency_dict = defaultdict(int)  # Dictionary to track frequency of response mapping keys
doc_count = 0  # Counter to track the total number of documents processed

for smartapi in SmartAPI.get_all(1000):
    doc_count += 1  # Increment the document count
    
    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)

    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        for edge in metakg:
            for map_key in edge['api']['bte']['response_mapping']:
                keys = edge['api']['bte']['response_mapping'][map_key].keys()
                
                # Remove all capital words from keys
                keys = [key for key in keys if not re.fullmatch(r'[A-Z]+', key)]
                
                # Update the set with cleaned keys
                keys_found_no_map_full_set.update(keys)

                # Update the frequency of each response mapping key
                for key in keys:
                    response_mapping_frequency_dict[key] += 1

                # Store metadata for each SmartAPI document
                if smartapi._id not in mkg_non_trapi_dict:
                    mkg_non_trapi_dict[smartapi._id] = {}

                mkg_non_trapi_dict[smartapi._id]['response_mapping'] = {
                    'keys_found': keys, 
                    'key_ct': len(keys), 
                    'data': edge['api']['bte']['response_mapping']
                }



In [20]:
# Print or store the response mapping key frequencies alphabetically
for key in sorted(response_mapping_frequency_dict.keys()):
    count = response_mapping_frequency_dict[key]
    print(f" {key}: {count}")

# Print the total number of documents processed
print(f"Total number of documents processed: {doc_count}")


 CHEMBL.COMPOUND: 18
 ComplexPortal: 5
 ICD11: 10
 KEGG.PATHWAY: 1
 KEGG_COMPOUND: 11
 NCBIGene: 462
 PHARMGKB.DISEASE: 6
 PHARMGKB.DRUG: 11
 PHARMGKB.GENE: 7
 PHARMGKB.PATHWAYS: 4
 PUBCHEM.COMPOUND: 14
 PUBCHEM_COMPOUND: 11
 TTD.DRUG: 10
 TTD.TARGET: 4
 UniProtKB: 37
 biolink:OMOP: 24
 biolink:chi_squared_statistic: 8
 biolink:evidence_count: 2
 biolink:frequency_qualifier: 2
 biolink:has_count: 26
 biolink:has_quotient: 10
 biolink:has_total: 10
 biolink:max_research_phase: 2
 biolink:name: 24
 biolink:p_value: 8
 biolink:supporting_text: 5029
 edge-attributes: 292
 evidence: 3
 foodb.food: 1
 inTaxon: 1
 input_name: 5434
 name: 7
 ncats.bioplanet: 2
 orphanet: 2
 output_name: 5459
 pubmed: 3
 qualifiers: 3
 ref_clinicaltrials: 8
 ref_doi: 4
 ref_isbn: 2
 ref_pmcid: 8
 ref_pmid: 5125
 ref_url: 23
 source_url: 9
 trapi_sources: 306
Total number of documents processed: 271


In [23]:
# keys_found_no_map_full_set

{'BIOCARTA',
 'CAS',
 'CHEBI',
 'CHEMBL.COMPOUND',
 'CL',
 'ComplexPortal',
 'DBSNP',
 'DOID',
 'DRUGBANK',
 'EFO',
 'ENSEMBL',
 'FB',
 'FOODON',
 'GO',
 'HGNC',
 'HMDB',
 'HP',
 'ICD11',
 'INCHIKEY',
 'KEGG',
 'KEGG.PATHWAY',
 'KEGG_COMPOUND',
 'LOINC',
 'MEDDRA',
 'MESH',
 'MGI',
 'MONDO',
 'MP',
 'NCBIGENE',
 'NCBIGene',
 'NCIT',
 'OMIM',
 'ORPHANET',
 'PHARMGKB.CHEMICAL',
 'PHARMGKB.DISEASE',
 'PHARMGKB.GENE',
 'PHARMGKB.PATHWAYS',
 'PUBCHEM.COMPOUND',
 'PUBCHEM_COMPOUND',
 'REACT',
 'RGD',
 'RHEA',
 'SGD',
 'SNOMEDCT',
 'TTD.DRUG',
 'TTD.TARGET',
 'UBERON',
 'UMLS',
 'UNII',
 'UniProtKB',
 'WB',
 'WIKIPATHWAYS',
 'ZFIN',
 'biolink:OMOP',
 'biolink:chi_squared_statistic',
 'biolink:evidence_count',
 'biolink:frequency_qualifier',
 'biolink:has_count',
 'biolink:has_quotient',
 'biolink:has_total',
 'biolink:max_research_phase',
 'biolink:name',
 'biolink:p_value',
 'biolink:supporting_text',
 'edge-attributes',
 'evidence',
 'foodb.food',
 'inTaxon',
 'input_name',
 'name',
 'ncats

In [42]:
pprint.pprint(mkg_non_trapi_dict['edeb26858bd27d0322af93e7a9e08761'])

{'query_operation': {'data': {'method': 'post',
                              'params': {'fields': 'associatedWith.mentions.diseases.mesh, '
                                                   'associatedWith.figureUrl,associatedWith.pmc',
                                         'size': 1000},
                              'path': '/query',
                              'request_body': {'body': {'q': '{{ queryInputs '
                                                             '}}',
                                                        'scopes': 'associatedWith.mentions.genes.ncbigene'}},
                              'server': 'https://biothings.ncats.io/pfocr',
                              'support_batch': True},
                     'key_ct': 12,
                     'keys_found': ['name',
                                    'smartapi',
                                    'tags',
                                    'x-translator',
                                    'provided_by

In [49]:
for key_id in mkg_non_trapi_dict:
    if "source_url" in mkg_non_trapi_dict[key_id]['response_mapping']['keys_found']:
        print(key_id)
    

38e9e5169a72aee3659c9ddba956790d


---

In [1]:
# Import libraries
import yaml
import warnings
import pprint

from controller.smartapi import SmartAPI
from utils.metakg import parser


# Ignore all warnings
warnings.filterwarnings("ignore")


In [2]:
mkg_parser = parser.MetaKGParser()

In [3]:
for smartapi in SmartAPI.get_all(10):

    raw_data = smartapi.raw.decode('utf-8')
    source_data = yaml.safe_load(raw_data)
    # Get non-TRAPI metakg data
    metakg = mkg_parser.get_non_TRAPI_metadatas(data=source_data)

    if metakg:
        print(smartapi._id)
        # print(vars(smartapi))  # This will print all attributes of the object

        # pprint.pprint(source_data)

        # for edge in metakg:
        #     print(f"--------------METAKG({smartapi._id})----------------")
        #     print(smartapi._id)
        #     print(source_data)


0991bc29489a8d918ff6035fd38b0ccc
1dad992a6ce8f680e59a5ea09d90670d
dc91716f44207d2e1287c727f281d339
04098a30dd6cb153b049712a7924ffff
23f770568b92b7a82063989b3ddd9706
e8818d98f59fb98ad5ba6e4637b5dec4
edeb26858bd27d0322af93e7a9e08761
edeb26858bd27d0322af93e7a9e08761
4153c947a32e2e2a55a320d0bee22077
78f324f4ac6b9fafe9085d68c332354c
15bf82dcf5bdd2c95d8687250b662cbb


---